In [1]:
path = './zomato.csv'

In [2]:
import csv
import os
import glob
import numpy as np
import math
from sklearn.metrics import accuracy_score, classification_report

/Users/indraaristya/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/indraaristya/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/indraaristya/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/indraaristya/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
def convert(sentence):
    import re
    try:
        highpoints = re.compile(u'([\U00002600-\U000027BF])|([\U0001f300-\U0001f64F])|([\U0001f680-\U0001f6FF])')
    except re.error:
        highpoints = re.compile(u'([\u2600-\u27BF])|([\uD83C][\uDF00-\uDFFF])|([\uD83D][\uDC00-\uDE4F])|([\uD83D][\uDE80-\uDEFF])')

    cleanEmoticon = highpoints.sub(u'\u25FD', sentence)
    return cleanEmoticon

def cleaning(sentence):
    sentence = sentence.replace('\ufeff','')
    sentence = sentence.replace('\n',' ')
    sentence = sentence.replace('\r',' ')
    sentence = sentence.replace('\n\n',' ')
    sentence = sentence.replace('\r\r',' ')
    sentence = sentence.replace('.',' ')
    sentence = sentence.replace(',',' ')
    sentence = sentence.replace('"',' ')
    sentence = sentence.replace('!',' ')
    sentence = sentence.replace('?',' ')
    sentence = sentence.replace('/',' ')
    sentence = sentence.replace('-',' ')
    sentence = sentence.lstrip()
    return convert(sentence.lower())


def count_word_in_class(x,y):
    count = {}
    count_word = {}
    word_class = {}
    for i in range(0, len(x)):
        x_split = x[i].split()
        for word in x_split:
            w_c = word+str(",")+str(y[i])
            if (w_c in count):
                count[w_c] += 1
            else:
                count[w_c] = 1
                
            if (word in count_word):
                count_word[word] += 1
            else:
                count_word[word] = 1
                
            if (y[i] in word_class):
                word_class[y[i]] += 1
            else:
                word_class[y[i]] = 1
    return count, count_word, word_class

def count_class_in_doc(y):
    count = {}
    for i in range(0, len(y)):
        if (y[i] in count):
            count[y[i]] += 1
        else:
            count[y[i]] = 1
    return count

def hitung_prob(count,word,uniq):
    return ((count + 1)/(word+uniq))

In [4]:
x = []
y = []
with open(path) as a:
    b = csv.reader(a)
    for row in b:
        x.append(cleaning(row[2]))
        y.append(row[3])
x = np.array(x)
y = np.array(y)

In [5]:
randomize = np.arange(len(x))
np.random.shuffle(randomize)
x = x[randomize]
y = y[randomize]

In [6]:
x_train = x[:160]
y_train = y[:160]

x_test = x[160:]
y_test = y[160:]

In [7]:
count_word_on_class, count_word, word_class = count_word_in_class(x_train,y_train)

In [8]:
count_class = count_class_in_doc(y_train)

In [9]:
kelaz = list(count_class.keys())
y_pred = []
for i in range(0,len(x_test)):
    prob_kelas = {}
    for j in count_class.keys():
        prob_c = 1
        x_split = x_test[i].split()
        for k in x_split:
            tag = k+str(',')+str(j)
            prob_c += math.log(hitung_prob(count_word_on_class.get(tag,0),word_class.get(j),len(count_word.keys())))
        nama = str(i)+','+str(j)
        prob_kelas[nama] = prob_c
    y_pred.append(kelaz[np.argmax(list(prob_kelas.values()))])
    

In [10]:
accuracy_score(y_test, y_pred)

0.8

In [11]:
print(classification_report(y_pred, y_test))

             precision    recall  f1-score   support

          0       0.69      0.79      0.73        14
          1       0.88      0.81      0.84        26

avg / total       0.81      0.80      0.80        40



In [12]:
# y_test

In [13]:
# y_pred

### Coba KFold Validation

In [43]:
from sklearn.model_selection import KFold

x = []
y = []
x_train = []
y_train = []
x_test = []
y_test = []
with open(path) as a:
    b = csv.reader(a)
    for row in b:
        x.append(cleaning(row[2]))
        y.append(row[3])
x = np.array(x)
y = np.array(y)
        
randomize = np.arange(len(x))
np.random.shuffle(randomize)

acc = []
folds = 4
kf = KFold(n_splits=folds)
for train, test in kf.split(x):
    x_train, x_test, y_train, y_test = x[train],x[test],y[train],y[test]
    
    count_word_on_class, count_word, word_class = count_word_in_class(x_train,y_train)
    count_class = count_class_in_doc(y_train)
    kelaz = list(count_class.keys())
    y_pred = []
    for i in range(0,len(x_test)):
        prob_kelas = {}
        for j in count_class.keys():
            prob_c = 1
            x_split = x_test[i].split()
            for k in x_split:
                tag = k+str(',')+str(j)
                prob_c += math.log(hitung_prob(count_word_on_class.get(tag,0),word_class.get(j),len(count_word.keys())))
            nama = str(i)+','+str(j)
            prob_kelas[nama] = prob_c
        y_pred.append(kelaz[np.argmax(list(prob_kelas.values()))])
    acc.append(accuracy_score(y_test, y_pred))
print("Mean Acc of",folds,"Fold:",np.mean(acc))

Mean Acc of 4 Fold: 0.745


In [15]:
print(classification_report(y_pred, y_test))

             precision    recall  f1-score   support

          0       0.93      0.48      0.64        29
          1       0.57      0.95      0.71        21

avg / total       0.78      0.68      0.67        50



## Data Sendiri

In [53]:
x_test = ['Saya suka sekali tempat dan suasananya, tapi makanannya biasa saja',
         'Makanannya enak sekali, rasanya pas, pokoknya top!',
         'Boleh juga makanannya, tapi pelayanannya biasa saja',
         'Makanan oke, pelayanannya juga oke',
         'tempatnya enak buat nongkrong ada 2 pilihan tempat outdor atau indor nama makanan dan minuman nya unik unik tapi ngeribetin pelayananya juga kurang ketika saya pesan minuman gelasnya yang terbuat dari kaca bocor dan minuman saya yang tumpah tidak di ganti saya pesan nasi goreng ldr rasanya tidak sebanding dengan harganya tempat nya kurang terang kalau buat foto foto hasilnya ga bagus seharusnya kualitas nya ditingkatkan lagi sesuai harga makanan dan minumannya']
kelaz = list(count_class.keys())
y_pred = []
for i in range(0,len(x_test)):
    prob_kelas = {}
    for j in count_class.keys():
        prob_c = 1
        x_split = x_test[i].split()
        for k in x_split:
            tag = k+str(',')+str(j)
            prob_c += math.log(hitung_prob(count_word_on_class.get(tag,0),word_class.get(j),len(count_word.keys())))
        nama = str(i)+','+str(j)
        prob_kelas[nama] = prob_c
    if (kelaz[np.argmax(list(prob_kelas.values()))] == '1'):
        y_pred.append('Positif')
    else:
        y_pred.append('Negatif')

In [55]:
for i in range(len(x_test)):
    print(x_test[i].strip(),' - ',y_pred[i])
    print()

Saya suka sekali tempat dan suasananya, tapi makanannya biasa saja  -  Negatif

Makanannya enak sekali, rasanya pas, pokoknya top!  -  Positif

Boleh juga makanannya, tapi pelayanannya biasa saja  -  Negatif

Makanan oke, pelayanannya juga oke  -  Positif

tempatnya enak buat nongkrong ada 2 pilihan tempat outdor atau indor nama makanan dan minuman nya unik unik tapi ngeribetin pelayananya juga kurang ketika saya pesan minuman gelasnya yang terbuat dari kaca bocor dan minuman saya yang tumpah tidak di ganti saya pesan nasi goreng ldr rasanya tidak sebanding dengan harganya tempat nya kurang terang kalau buat foto foto hasilnya ga bagus seharusnya kualitas nya ditingkatkan lagi sesuai harga makanan dan minumannya  -  Negatif

